In [ ]:
from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）をセット
DATASET_SLUG = 'gns-codes'  # 変更する場合はここを更新
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 取得優先順位: 展開済み code/ があればそれを使う。無ければ code.zip を展開。どちらも無ければエラー。
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code ディレクトリも code.zip も見つかりません。Add Data で {DATASET_SLUG} を追加してください。")

if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
%cd $repo_dir
# 依存パッケージは Kaggle 環境の標準ライブラリを利用（追加インストールなし）


In [ ]:
!pip install torch-geometric 

In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'hamiltonian_sph'
cfg['amp_enable'] = False
cfg['ntraining_steps'] = 100
cfg['train_dataset_count'] = 10
cfg['valid_dataset_count'] = 2
cfg['validation_interval'] = 10

# Kaggle 入力データセットのパスに合わせて上書き
dataset_root = '/kaggle/input/dam-break-left/dam_break_left'
cfg['data_path'] = dataset_root
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/working/code
# GPUを2枚使う設定
!torchrun --nproc_per_node=2 src/train.py --config config.yaml


In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'hamiltonian_sph'
cfg['rollout_inference_max_examples'] = 1

# 推論時のデータセットパスを上書き
dataset_root = '/kaggle/input/dam-break-left/dam_break_left'
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/working/code
!python src/train.py --config config_rollout.yaml
!python analyze_rollouts.py
!python visualize_rollout.py
